In [ ]:
from keras.models import Model
from keras.datasets import cifar10
from keras.layers.merge import concatenate
from keras.callbacks import ModelCheckpoint
from keras.layers.convolutional import Conv2D
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input

In [ ]:
# load dataset
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
print('Train: X=%s, y=%s' % (X_train.shape, Y_train.shape))
print('Test: X=%s, y=%s' % (X_test.shape, Y_test.shape))

In [ ]:
# Change the labels from integer to categorical data
y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)

In [ ]:
def composite_function(ip, n_filter, bottleneck=False, dropout_rate=None):

  x = BatchNormalization()(ip)
  x = Activation('relu')(x)

  # Bottleneck layers
  if bottleneck:
    x = Conv2D(n_filter * 4, kernel_size=(1, 1), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
  
  x = Conv2D(n_filter, kernel_size=(3, 3), padding='same', use_bias=False)(x)

  # Dropout layer after each convolutional layer except the first one.
  if dropout_rate:
    x = Dropout(dropout_rate)(x)

  return x

In [ ]:
def dense_block(x, n_layers, n_filter, growth_rate, bottleneck=False, dropout_rate=None):

  for i in range(n_layers):
    cf = composite_function(x, n_filter, bottleneck, dropout_rate)
    x = concatenate([x, cf])

    n_filter += growth_rate

  return x, n_filter

In [ ]:
def transition_block(ip, n_filter, compression=0.5):

  x = BatchNormalization()(ip)
  x = Activation('relu')(x)
  x = Conv2D(int(n_filter * compression), kernel_size=(1, 1), padding='same', use_bias=False)(x)
  x = AveragePooling2D((2, 2), strides=(2, 2))(x)

  return x

In [ ]:
def DenseNet(input_shape, n_dense_block=3, growth_rate=12, n_filter=-1, n_layers_per_block=-1, bottleneck=False, compression=0.0, dropout_rate=0.0, activation='softmax', n_classes=10):
  
  img_input = Input(shape=input_shape)

  if n_filter == -1:
    n_filter = 2 * growth_rate
  
  n_layers=0
  final_n_layer=0
  if n_layers_per_block == -1:
    count = 12
    if bottleneck:
      count = count // 2
    n_layers = [count for _ in range(n_dense_block)]
    final_n_layer = count    
  else:
    n_layers = [n_layers_per_block for _ in range(n_dense_block)]
    final_n_layer = n_layers_per_block

  initial_kernel = (3,3)
  initial_strides = (1, 1)

  x = Conv2D(n_filter, initial_kernel, padding='same', strides=initial_strides, use_bias=False)(img_input)

  for block_idx in range(n_dense_block - 1):
    x, n_filter = dense_block(x, n_layers[block_idx], n_filter, growth_rate, bottleneck=bottleneck, dropout_rate=dropout_rate)
    x = transition_block(x, n_filter, compression=compression)
    n_filter = int(n_filter * compression)

  # The last dense_block does not have a transition_block
  x, n_filter = dense_block(x, final_n_layer, n_filter, growth_rate, bottleneck=bottleneck, dropout_rate=dropout_rate)

  # x = BatchNormalization()(x)
  # x = Activation('relu')(x)
  x = GlobalAveragePooling2D()(x)
  x = Dense(n_classes, activation=activation)(x)

  model = Model(img_input, x)

  return model

In [ ]:
checkpoint_name = 'Weights-{epoch:03d}--{accuracy:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='accuracy', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

## Without Bottleneck

In [ ]:
model = DenseNet((32, 32, 3), n_dense_block=3, growth_rate=12, bottleneck=False, compression=0.5)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics= ['accuracy'])

In [ ]:
history_wb = model.fit(X_train, y_train, epochs=40, batch_size=64, validation_split = 0.2, callbacks=callbacks_list, workers=-1)

In [ ]:
loss_wb, acc_wb = model.evaluate(X_test, y_test)
display(loss_wb, acc_wb)

## With Bottleneck

In [ ]:
model = DenseNet((32, 32, 3), n_dense_block=3, growth_rate=12, bottleneck=True, compression=0.5)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics= ['accuracy'])

In [ ]:
history_b = model.fit(X_train, y_train, epochs=40, batch_size=64, validation_split = 0.2, callbacks=callbacks_list, workers=-1)

In [ ]:
loss_b, acc_b = model.evaluate(X_test, y_test)
display(loss_b, acc_b)

## Dropout rate 20%

In [ ]:
model = DenseNet((32, 32, 3), n_dense_block=3, growth_rate=12, bottleneck=True, compression=0.5, dropout_rate=0.2)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics= ['accuracy'])

In [ ]:
history_drop = model.fit(X_train, y_train, epochs=40, batch_size=64, validation_split = 0.2, callbacks=callbacks_list, workers=-1)

In [ ]:
loss_drop, acc_drop = model.evaluate(X_test, y_test)
display(loss_drop, acc_drop)

## Data Augmentation

 Best Performance

In [ ]:
model = DenseNet((32, 32, 3), n_dense_block=3, growth_rate=12, bottleneck=True, compression=0.5)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics= ['accuracy'])

In [ ]:
generator = ImageDataGenerator(rotation_range=15, width_shift_range=5./32, height_shift_range=5./32, horizontal_flip=True)
generator.fit(X_train, seed=0)

In [ ]:
history_aug = model.fit_generator(generator.flow(X_train, y_train, batch_size=64),
                    steps_per_epoch=len(X_train) // 64, epochs=40,
                    callbacks=callbacks_list,
                    verbose=1, workers=-1)

In [ ]:
loss_aug, acc_aug = model.evaluate(X_test, y_test)
display(loss_aug, acc_aug)